In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [8]:
#df = pd.read_csv('../input/UCI_Credit_Card.csv')
df = pd.read_csv('dataset.csv')

df.columns = df.iloc[0]
df.reindex(df.index.drop(0))
df = df.drop(0)

df.columns.values

X = df.iloc[:,1:24]
Y = df['default payment next month']
#Y = df['Y']

df.head()
#df.columns.values

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [9]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

corr=df.corr()
corr = (corr)
print(corr)
#plt.figure(figsize=(20,20))
#sns.heatmap(corr, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 13},
#            xticklabels=corr.columns.values,
#            yticklabels=corr.columns.values)
#plt.title('Heatmap of Correlation Matrix')

Empty DataFrame
Columns: []
Index: []


In [17]:
from sklearn.preprocessing import OneHotEncoder
OHE = OneHotEncoder(sparse=False,categories='auto')
sex = OHE.fit_transform(X[['SEX']])
marriage = OHE.fit_transform(X[['MARRIAGE']])
education = OHE.fit_transform(X[['EDUCATION']])

cat_variables = np.hstack((sex, marriage, education))

cat_var_names = ['SEX','MARRIAGE', 'EDUCATION']

num_variables = X.drop(cat_var_names, axis=1)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(num_variables)
stdz_num_variables = scaler.transform(num_variables)

final_X = np.hstack((cat_variables,stdz_num_variables))

<class 'numpy.ndarray'>


/home/alain/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/alain/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  from ipykernel import kernelapp as app


In [11]:
from sklearn.feature_selection import SelectKBest

test = SelectKBest(k=10)
fit = test.fit(final_X,Y)
np.set_printoptions(precision=1)
print(fit.scores_)
features = fit.transform(final_X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, Y, test_size=0.33, random_state=42)

[4.8e+01 4.8e+01 5.2e+00 2.7e+01 2.8e+01 2.9e+00 4.0e+00 7.9e+01 4.0e+01
 3.2e+01 1.9e+01 4.0e+01 1.2e+00 7.2e+02 5.8e+00 3.5e+03 2.2e+03 1.8e+03
 1.5e+03 1.3e+03 1.1e+03 1.2e+01 6.0e+00 5.9e+00 3.1e+00 1.4e+00 8.7e-01
 1.6e+02 1.0e+02 9.5e+01 9.7e+01 9.1e+01 8.5e+01]


In [13]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

estimator = SVC(kernel='rbf',gamma=50)
selector = estimator.fit(X_train,y_train)
#print(X_train.columns[selector.support_])
y_predict_test = selector.predict(X_test)
y_predict_train = selector.predict(X_train)
print(accuracy_score(y_train, y_predict_train))
print(accuracy_score(y_test, y_predict_test))

0.895223880597015
0.7986868686868687


In [16]:
from ipynb.fs.full.costo import custom_cost

custom_cost(selector, X_train, y_train)

/home/alain/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1037: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if pos_label not in present_labels:


ValueError: pos_label=1 is not a valid label: array(['0', '1'], dtype='<U1')